In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime
from datetime import timedelta, date
from pandas.tseries.offsets import BDay

In [ ]:
# Change the excel file link every time before running the algorithm further (also change the path below)
df = pd.read_excel("/content/52weekhighNSE.xlsx")
data = df['Symbol']
stocks = data.tolist()

In [ ]:
stock_symbols = []
for stock in stocks:
  symbol = stock + '.NS'
  stock_symbols.append(symbol)

In [ ]:
today = datetime.datetime.now()
ndays = datetime.timedelta(days = 1)
date = (today - ndays).date()
end_date = date.strftime("%Y-%m-%d")
date_range = pd.date_range(end=end_date, periods=365)
trading_dates = date_range.date.tolist()
start_date = trading_dates[0]
stock_dict = {}

for symbol in stock_symbols:
    try:
        data = yf.download(symbol, start=start_date, end=end_date, interval='1d')
        if not data.empty:
            stock_dict[symbol] = data
        else:
            print(f"WARNING: {symbol}: Data is empty, skipping.")
    except Exception as e:
        print(f"ERROR: {symbol}: {e}")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [ ]:
for stock, df in stock_dict.items():
  print(stock)
  print(df)
  print("="*50)

ACE.NS
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2022-10-31  303.799988  313.799988  303.049988  308.549988  308.156128   
2022-11-01  310.000000  313.799988  304.799988  306.399994  306.008881   
2022-11-02  306.399994  307.649994  292.200012  299.450012  299.067780   
2022-11-03  297.950012  303.950012  294.899994  298.649994  298.268768   
2022-11-04  298.600006  330.000000  298.600006  327.750000  327.331635   
...                ...         ...         ...         ...         ...   
2023-10-20  711.950012  756.599976  708.549988  739.099976  739.099976   
2023-10-23  746.500000  762.000000  705.000000  714.150024  714.150024   
2023-10-25  719.000000  732.950012  697.000000  702.900024  702.900024   
2023-10-26  700.049988  719.450012  657.299988  715.750000  715.750000   
2023-10-27  719.000000  755.000000  719.000000  750.650024  750.650024   

             Volume  
Date    

In [ ]:
results = []

for stock, data in stock_dict.items():

  # 52 WEEK ALL TIME HIGH
  stock_yf = yf.Ticker(stock)
  # fifty_two_week_high = stock_yf.info["fiftyTwoWeekHigh"]

  # RECENT CLOSE
  today = datetime.datetime.now()
  ndays = datetime.timedelta(days = 1)
  date = today - ndays
  end_date = date.date()

  recent_close = round(data['Close'].iloc[-1], 2)

  # REFERENCE VALUE CALCULATIONS

  # DATA FOR REFERENCE VALUE
  end_date_reference = today - datetime.timedelta(days=5)
  end_date_reference = pd.to_datetime(end_date_reference)
  data_filtered = data[data.index <= end_date_reference]

  # REFERENCE VALUE
  highest_close_candle = data_filtered[data_filtered['Close'] == data_filtered['Close'].max()]
  highest_open_candle = data_filtered[data_filtered['Open'] == data_filtered['Open'].max()]

  # Check if data_filtered is not empty
  if not data_filtered.empty:
      # Calculate highest_close_candle and highest_open_candle
      highest_close_candle = data_filtered[data_filtered['Close'] == data_filtered['Close'].max()]
      highest_open_candle = data_filtered[data_filtered['Open'] == data_filtered['Open'].max()]

      if not highest_close_candle.empty and not highest_open_candle.empty:
          if highest_close_candle['Close'].values[0] > highest_open_candle['Open'].values[0]:
              # If the highest closing price is used, store the reference value and date
              reference_value = round(highest_close_candle['Close'].values[0], 2)
              reference_date = highest_close_candle.index[0]
          else:
              # If the highest opening price is used, store the reference value and date
              reference_value = round(highest_open_candle['Open'].values[0], 2)
              reference_date = highest_open_candle.index[0]

  # DAYS SINCE REFERENCE VALUE
  end_date = pd.to_datetime(end_date)
  days_since_reference_day = (end_date - reference_date).days

  # MAX DIP CALCULATIONS

  # Find the rows starting from the reference date
  data_after_reference = data[data.index >= reference_date]

  # Find the minimum of 'Open' and 'Close' prices after the reference date
  min_open_or_close = min(data_after_reference['Open'].min(), data_after_reference['Close'].min())

  # Find the day (date) of the lowest open or close
  min_open_or_close_day = data_after_reference[(data_after_reference['Open'] == min_open_or_close) | (data_after_reference['Close'] == min_open_or_close)].index[0].date()
  min_open_or_close_day = pd.to_datetime(min_open_or_close_day)

  # Assign max_dip as the minimum of 'Open' and 'Close' prices
  max_dip = round(min_open_or_close, 2)

  # Days since max dip
  days_since_max_dip = (end_date - min_open_or_close_day).days


  # CROSSED DAYS CALCULATIONS

  # Number of Days where the High of the Day was more than the Reference Value
  crossed_days = data[data.index > reference_date]
  crossed_days = crossed_days[crossed_days['High'] > reference_value]
  num_crossed_days = len(crossed_days)

  results.append([stock, recent_close, reference_value, reference_date, days_since_reference_day, max_dip, min_open_or_close_day, days_since_max_dip, num_crossed_days])

# Final DataFrame
res_df = pd.DataFrame(results, columns=['Stock Symbol',  'Recent Close', 'Ref. Val', 'Ref. Date', 'Days Since Ref. Val', 'Max Dip', 'Max Dip Day', 'Days Since Max Dip', 'No. Days Threshold Breached'])

In [ ]:
res_df

,Stock Symbol,Recent Close,Ref. Val,Ref. Date,Days Since Ref. Val,Max Dip,Max Dip Day,Days Since Max Dip,No. Days Threshold Breached
0,ACE.NS,750.65,799.00,2023-08-18,73,669.05,2023-10-04,26,0
1,ANANTRAJ.NS,241.10,241.00,2023-10-17,13,223.55,2023-10-23,7,3
2,ANGELONE.NS,2535.80,2413.90,2023-10-26,4,2294.00,2023-10-26,4,1
3,ANUP.NS,2136.65,2279.95,2023-08-02,89,1949.50,2023-09-28,32,1
4,APOLLO.NS,74.85,78.45,2023-10-20,10,68.05,2023-10-23,7,0
5,ARROWGREEN.NS,449.00,444.00,2023-10-20,10,381.40,2023-10-26,4,1
6,ARVIND.NS,175.55,193.90,2023-10-18,12,166.65,2023-10-26,4,1
7,ASIANTILES.NS,78.00,81.75,2023-10-20,10,73.30,2023-10-23,7,1
8,AVADHSUGAR.NS,704.65,704.80,2023-10-26,4,686.00,2023-10-26,4,1
9,BLUESTARCO.NS,863.35,1518.40,2023-06-13,139,704.70,2023-08-25,66,0
